# Programming Assignment 1

Matriculation number: 01/1153929

In [1]:
import numpy as np
import csv

### No. 1

In [2]:
def var_model(data, p: int, intercept: bool):
    """
    Function to compute coefficients, corresponding t-ratios and residual covariance matrix based on a VAR model
    
    Note: this function displays the values of the coefficients and t-ratios in a different order 
    then the build-in function does:
    
          Order here:  intercept for y1, L1.y1, L2.y1, ..., L1.y2, L2,y2...
                       intercept for y2, L2.y1, L2.y1, ..., L1.y2, L2,y2...
                       ... 
    """
    
    
    "Preparation of variables for calculation"
    # define Z
    num_rows, num_cols  = data.shape
    Z=[]
    if intercept==1:
        first_row = [1] * (num_cols-p)
        Z.append(first_row)
    for i in range(0, num_rows):
        z = data[i]
        for i in reversed(range(0,p)):
            z_1 = z[i:-(p-i)]
            Z.append(z_1)
    Z = np.array(Z)
    
    # create new Y without presample values
    Y_new = []
    for i in range(0, num_rows):
        y = data[i]
        y = y[p:]
        Y_new.append(y)
    Y_new = np.array(Y_new)
    
    
    "Compute results"
    # coefficients
    B_hat = Y_new@Z.T@np.linalg.inv(Z@Z.T)
    
    # estimated residual covariance matrix 
    Y_hat = B_hat@Z
    U_hat = Y_new - Y_hat
    K, T = U_hat.shape
    eu_1 = 1/(T-K*p-1)
    eu_2 = eu_1 * U_hat
    E_u = eu_2@U_hat.T
    
    # calculate t-ratios
    t_1 = np.linalg.inv(Z@Z.T)
    t_2 = np.kron(t_1, E_u)
    dia = np.diagonal(t_2)
    dia_sq = np.sqrt(dia)
    b = B_hat.flatten('F')     # vectorize B_hat
    b2 = np.matrix(B_hat)
    b2 = b2.flatten()
    b3 = np.squeeze(np.asarray(b2))
    t = np.divide(b, dia_sq)
    t = t.reshape(K, B_hat.shape[1], order= "F")
    
    return B_hat, t, E_u    

### No.2

In [3]:
## functions to prepare data set

# load data set, delete dates and convert to array
def convert_data(x):    
    with open(f'{x}.csv', 'r') as file:
        reader = csv.reader(file)
        y=[]
        [y.append(row[1]) for row in reader]

    del y[0]
    y = list(map(float, y))
    y = np.array(y)
    return y

# compute growth rate
def quat_gr(x):
    x = np.log(x)
    x = np.diff(x)
    x = 100 * x
    return x

In [4]:
# use function to actually prepare data set
FFR = convert_data('FEDFUNDS(1)')
FFR = FFR[1:] # remove one value to garantee equal length of time series
GDP = convert_data('GDPC1')
GDP_diff = quat_gr(GDP)
DEF = convert_data('GDPDEF')
DEF_diff = quat_gr(DEF)

In [5]:
# create Y
Y = np.c_[GDP_diff, DEF_diff, FFR].T

In [6]:
Y.shape

(3, 250)

In [7]:
# use VAR function
coeff, t_ratios, cov_matrix = var_model(Y, 4, 1)

In [8]:
coeff

array([[ 0.73841693, -0.00136472,  0.13106845,  0.06634204,  0.01712105,
        -0.20967773,  0.23637034, -0.18850331, -0.04983365,  0.10475077,
        -0.25591333,  0.05900275,  0.09494658],
       [ 0.07707766, -0.01731531,  0.01606932,  0.0010667 ,  0.00337405,
         0.56376804,  0.14202479,  0.13904395,  0.08469262,  0.05487985,
        -0.02622414, -0.02223694, -0.00903667],
       [-0.26140514,  0.22121981,  0.10224683,  0.06151369,  0.03990731,
         0.10610175,  0.76614749, -0.28244781, -0.14819337,  0.79145067,
        -0.0780109 ,  0.36828521, -0.16742993]])

In [9]:
t_ratios

array([[ 4.35761667, -0.02016449,  1.90556797,  0.97262942,  0.26599918,
        -0.74443971,  0.72720731, -0.57046516, -0.17105643,  1.57514908,
        -3.14737037,  0.71585058,  1.39234064],
       [ 1.8442399 , -1.03732317,  0.94725166,  0.0634079 ,  0.21254117,
         8.11557258,  1.7716214 ,  1.70609703,  1.17870115,  3.34594518,
        -1.3076688 , -1.0938725 , -0.53729899],
       [-1.54481965,  3.27327546,  1.4886494 ,  0.90312247,  0.62089573,
         0.37723856,  2.36044526, -0.85598204, -0.50940327, 11.9180324 ,
        -0.96078571,  4.47456406, -2.45875664]])

In [10]:
cov_matrix

array([[1.17169863, 0.05667958, 0.26128313],
       [0.05667958, 0.0712743 , 0.04153689],
       [0.26128313, 0.04153689, 1.16837793]])

### Extra: Result check

In [ ]:
from statsmodels.tsa.api import VAR

C:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [ ]:
model = VAR(Y.T)
results = model.fit(4)

In [ ]:
results.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Wed, 09, Nov, 2022
Time:                     21:29:23
--------------------------------------------------------------------
No. of Equations:         3.00000    BIC:                   -1.71865
Nobs:                     246.000    HQIC:                  -2.05061
Log likelihood:          -728.429    FPE:                   0.102892
AIC:                     -2.27437    Det(Omega_mle):       0.0881630
--------------------------------------------------------------------
Results for equation y1
           coefficient       std. error           t-stat            prob
------------------------------------------------------------------------
const         0.738417         0.169454            4.358           0.000
L1.y1        -0.001365         0.067680           -0.020           0.984
L1.y2        -0.209678         0.281658           -0.744           0.457
L1.y3         0.104751